First you need to link your Google Drive to the notebook in order to access the files needed for this module.

Run the cell below and follow instructions to mount the drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Installing Biopython

At the beginning of each module, we will install **Biopython**. Biopython is a large open-source application programming interface (API) used in both bioinformatics software development and in everyday scripts for common bioinformatics tasks. It contains several packages that you will need to import which will allow you to run the analyses required for this project. 

REF:
* Cock, P. J., Antao, T., Chang, J. T., Chapman, B. A., Cox, C. J., Dalke, A., Friedberg, I., Hamelryck, T., Kauff, F., Wilczynski, B., & de Hoon, M. J. (2009). Biopython: freely available Python tools for computational molecular biology and bioinformatics. Bioinformatics (Oxford, England), 25(11), 1422–1423. https://doi.org/10.1093/bioinformatics/btp163


In [ ]:
!pip install biopython

# Investigating the biological impact of the mutation and its possible role in human disease
For this section, your research will focus on investigating the biological impact of the mutation you are studying. To do this, you will use the OMIM and KEGG databases.

## OMIM Search for information on genetic diseases

The **OMIM** (Online Mendelian Inheritance of Man) database contains short, referenced reviews about genetic loci and genetic diseases. It
can be a very useful resource for finding out what type of research has been done on a gene or a disease.

REF:
* http://www.ncbi.nlm.nih.gov/entrez/query.fcgi?db=OMIM

## Install and import the necessary packages:

The **romim package** was created to query the OMIM database but it runs in R. 

**R** is another programming language so you will need to install **rpy2** to run R code in Google Colab.

**Methods** and **remotes** are R packages that help us both install the package and use the functions in the code.

The **XML** package will be used to read the results that you obtain from your database searches. 

REF:
* https://github.com/davetang/romim

In [ ]:
# Install the rpy2 interface to run R code
%load_ext rpy2.ipython

In [ ]:
%%R # This must precede all R code in Colab, to allow R code to run 

# Installing the main package
# Note how different it is from Python code
remotes::install_github('davetang/romim')

# Import the library associated with the package
library(romim)

# Intalling several packages
install.packages('XML')
install.packages('methods')
install.packages("remotes")

# Press 1 and ENTER when prompted

## Obtaining the ID number (called mim number) associated with the CYP1A1 gene entry in OMIM

In [ ]:
%%R # This must precede all R code in Colab, to allow R code to run 

# To access OMIM, we will use this key which will work as our password to access the database
set_key('4PUvWRqSSD2BuprIVAP_VQ') 

# Write CYP1A1 in the parenthesis below to obtain the entry for CYP1A1
gene_to_omim('####', show_query=TRUE)


Now, the link above is a file with our information. Let's download and parse it.

In [ ]:
%%R
# Saving the file:

# Paste the url between the quotes below
url <- '#####'

# Set destination (write the file path and a file name followed by .xml)
destfile <- '/content/drive/MyDrive/Colab_Notebooks/p450_project_files/NAME THIS FILE.xml'

# Download the file
download.file(url, destfile)


In [ ]:
%%R
# Parse the file for readability
result <- xmlParse(file = 'WHERE IS THE FILE?')

# Read the file
read <- read_xml('WHERE IS THE FILE?')

# Find the mim Number
num <- xml_find_all(read, ".//mimNumber")

# Display the number
num

Write down the mim number for the entry from the results above.

Answer here

## Using OMIM to obtain more information about the gene
This time you will search the OMIM 'CYP1A1' entry for information.

The function 'get_omim' helps you do just that since we can set certain arguments to 'TRUE' and obtain specific information about the entry.

Run the next cell to see a list of Arguments that you can access.

In [ ]:
%%R
help(get_omim)

Start by reading some sections in the entry.

In [ ]:
%%R
set_key('KEY HERE') # The key must be added before every request

# Using mim number to get the article list
# Write the mim number inside the parenthesis and set 'referenceList' to 'TRUE'
omim_result <- get_omim(####, referenceList = ###)

# Save the results as an XML file
saveXML(omim_result, file='FILE NAME HERE.xml') # Write a file name

# File name will display was output for this cell


### Display the results in the form of a table

In [ ]:
#@title Load the results by providing the file name in this form (include file extension .xml)

# MAKING RESULTS LOOK GOOD
import xml.etree.ElementTree as ET
import csv
import pandas as pd

file_name = '' #@param {type:"string"}


tree = ET.parse(file_name)
root = tree.getroot()
 
Ref_data = open('refdata.csv', 'w')
 
csvwriter = csv.writer(Ref_data)
authors_head = []

 
count = 0
for member in root.findall('.//reference'):
    authors = []
    ref_list = []
    if count == 0:
        author = member.find('.//authors').tag
        authors_head.append(author)

        title = member.find('.//title').tag
        authors_head.append(title)

        source = member.find('.//source').tag
        authors_head.append(source)
        
        pubmed = member.find('.//pubmedID').tag
        authors_head.append(pubmed)
       
        csvwriter.writerow(authors_head)
        count = count + 1
    
    author = member.find('.//authors').text
    authors.append(author) 
    title = member.find('.//title').text
    authors.append(title)
    source = member.find('.//source').text
    authors.append(source)

    #pubmed = member.find('.//pubmedID')
    ementa = getattr(member.find('.//pubmedID'), 'text', None)
    authors.append(ementa)
         
    csvwriter.writerow(authors)
 
 
Ref_data.close()

data= pd.read_csv("refdata.csv")
data

## Reading the abstract from the article by Petersen, et al. in the list

In [ ]:
from Bio import Entrez
# Searching for the abstract in the database Pubmed
Entrez.email = 'YOUR EMAIL HERE'

query7 = Entrez.efetch(db='####', id = '####', rettype = '###', retmode = '###')

# Reading the query and printing it
print(query7.read())

# Closing the query
query7.close()

## Answer the following questions:##
Input your answer in the cell below each question and press SHIFT+ENTER.

1. What journal was this article published in?


Answer here

2. At what institution did these authors work?

Answer here

3. What compound do they call a “procarcinogen”?

Answer here

5. How is CYP1A1 related to the procarcinogen?

Answer here

# Obtain more information from the entry

In [ ]:
# Search OMIM again to obtain description information
%%R
set_key('KEY HERE')

# Using mim number to get the entry, set 'text' argument to true
omim_result <- get_omim(###, ### = ###)

# Save the xml to a file called molgen.xml
saveXML(omim_result, file="FILE NAME HERE.xml")

In [ ]:
#@title Load the results by providing the file name in this form (include file extension .xml)

# MAKING RESULTS LOOK GOOD
import xml.etree.ElementTree as ET
import csv
import pandas as pd

file_name = "" #@param {type:"string"}

tree = ET.parse(file_name)
root = tree.getroot()
 
Ref_data4 = open('refdata4.csv', 'w')
 
csvwriter = csv.writer(Ref_data4)
allele_head = []

 
count = 0
for member in root.findall('.//textSection'):
    allele = []
    ref_list = []

    if count == 0:
      des = member.find('.//textSectionTitle').tag
      allele_head.append(des)
      
      mut = member.find('.//textSectionContent').tag
      allele_head.append(mut)
     
      csvwriter.writerow(allele_head)
      count = count + 1
       
    des = member.find('.//textSectionTitle').text
    allele.append(des)
    
    mut = member.find('.//textSectionContent').text
    allele.append(mut)

    csvwriter.writerow(allele)
  
Ref_data4.close()

data4= pd.read_csv("refdata4.csv")
pd.set_option('display.max_colwidth',10000)

data4

## Answer the following questions:
Input your answer in the cell below each question and press SHIFT+ENTER.

Read the section above titled **Molecular Genetics**.

1. What did the Kouri article in 1982 find about CYP1A1 and lung cancer?

Answer here

2. What percentage of Caucasians has the highly inducible form of CYP1A1?
 

Answer here

3. What type of smoker is most susceptible to an increase risk for lung cancer due to inheriting the CYP1A1 Leu to Val polymorphism?


Answer here

4. What is the proposed mechanism for the increased cancer risk for people who have the valine-type CYP1A1 enzyme?


Answer here

5. What is the difference between “highly-inducible” and “enhanced catalytic activity” for an enzyme?

Answer here

6. How are the levels of PAH-DNA adducts affected by the exon 7 leu to val polymorphism?

Answer here